
## Basket Option on Two Underlying Stocks

Since we will have two dimensions to integrate over, the Quasi Monte Carlo will be a better alternative than the ordinary Monte Carlo methods. This method can handle these types of problems with greater accuracy and faster convergence than regular Monte Carlo, since the options payoff depens on smooth functions of the underlying assets. The QMC will be implemented with a Sobol sequence.

Low discrepancy sequences such as Sobol sequenses are designed to fill out the space of possible values more evenly than purely random samples. These types of sequences reduce the likelihood of missing important areas of the payoff distribution. This leads to more accurate estimates for the basket option pricing. The stocks will also be correlated and modelling this with a Sobol sequence can capture this relationship more accuratley than random smapling.




In [49]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm
from scipy.stats import qmc
from scipy.stats import pearsonr





r = 0.04 #risk-free interest rate


#Importing and modifying data
MSFT = yf.Ticker("MSFT")
META = yf.Ticker("META")

msft_data = MSFT.history(start="2015-01-01",end="2024-01-01")
meta_data = META.history(start="2015-01-01",end="2024-01-01")

msft_price = msft_data[["Close"]]
meta_price = meta_data[["Close"]]
msft_price.columns = ["MSFT"]
meta_price.columns = ["META"]

msft_returns = np.log(msft_price).diff().dropna().to_numpy()
meta_returns = np.log(meta_price).diff().dropna().to_numpy()

#Sample volatilities, can be expanded in future to more complex volatilities
msft_vol = np.std(msft_returns)*np.sqrt(252) #Rescaling to yearly volatility
meta_vol = np.std(meta_returns)*np.sqrt(252) #Rescaling to yearly volatility

#Calculating the correlation between the two stocks
corr, p = pearsonr(msft_returns,meta_returns)
#Creating correlation matrix
#If the p-value is less than 0.05, then there is significant correlation between the two stocks
#and this should be included in the model.
if p < 0.05:
    Sigma = np.array([[msft_vol**2,msft_vol*meta_vol*corr[0]],[msft_vol*meta_vol*corr[0], meta_vol**2]])
else:
    Sigma = np.array([[msft_vol**2, 0], [0, meta_vol**2]])

#Creating a 2-d Sobol sequence (since we have 2 assets)
sampler = qmc.Sobol(d=2, scramble=False)
#m is how big the sample should be (2^17) want around 100 000 samples
sobol = sampler.random_base2(m=17)
print(sobol.shape)

#Creating reasonable strike prices (70%,100%,130%) of current average price
latest_msft = msft_price.iloc[-1,0]
latest_meta = meta_price.iloc[-1,0]
avg_price = (latest_msft+latest_meta)/2
strikes = [np.floor(avg_price*x) for x in [0.7, 1, 1.3]]
print(strikes)
print(corr)
print(meta_vol,msft_vol)


(131072, 2)
[np.float64(254.0), np.float64(363.0), np.float64(472.0)]
[0.58337964]
0.3808224890498157 0.27802632365996954


The simulation of the two stock prices are done through calculating the Cholesky matrix, this is a matrix A such that
$$\Sigma = AA^{-1}$$
This allows us to calculate the price of two assets when they are correlated.
The payoff for the call option during each of the iterations is given by
$$C = (0.5(s_1 + s_2)-K, 0)^+$$
and for the put option by
$$P = (K - 0.5(s_1 + s_2), 0)^+$$
The finaly price for the option is the average payoff from all the iterations, discounted by the risk-free rate.

In [50]:


def qmcBasket(option, K, T, iterations):
    #Creating a Cholesky matrix to take into account the correlation
    A = np.linalg.cholesky(Sigma)
    h = np.empty(iterations)
    for i in range(iterations):
        Z1 = norm.ppf(sobol[i+1][0]) #"Random" variable for MSFT
        Z2 = norm.ppf(sobol[i+1][0]) #"Random" variable for META
        X = A @ [Z1, Z2]
        
        S_msft = latest_msft*np.exp((r-(msft_vol**2)/2)*T + np.sqrt(T)*X[0])
        S_meta = latest_meta*np.exp((r-(meta_vol**2)/2)*T + np.sqrt(T)*X[1])

        if option == "Call":
            #Payoff is the discounted payoff of the option
            h[i] = np.exp(-r*T)*np.maximum(((S_msft+S_meta)/2)-K, 0)    
        elif option == "Put":
            h[i] = np.exp(-r*T)*np.maximum(K-((S_msft+S_meta)/2), 0)    

    #Returning the average payoff
    return np.mean(h)


options = ["Call", "Put"]
T = 5 #Years to maturity
for option in options:
    for k in strikes:
        price = qmcBasket(option, k, T, 40000)
        print(f"The price for a {option} with strike {int(k)} is: ${round(price,3)}")
    print("")

The price for a Call with strike 254 is: $256.602
The price for a Call with strike 363 is: $208.009
The price for a Call with strike 472 is: $171.653

The price for a Put with strike 254 is: $29.149
The price for a Put with strike 363 is: $69.799
The price for a Put with strike 472 is: $122.684

